In [6]:
# Visualization configuration (tweakable knobs)
cfg = {
    # Data
    'year': 2017,
    'file_name': '../data/processed/simulations/simulations_apr7.parquet',

    # Layout sizing
    'target_width': 1000,
    'margins': {'l': 110, 'r': 210, 't': 130, 'b': 90},
    'spacing': {'horizontal': 0.004, 'vertical': 0.004},
    'spacing_mode': 'auto-match',  # 'fixed' uses values above, 'auto-match' aligns subplot gaps to cell gaps

    # Heatmap look
    'colorscale': 'Viridis',
    'xgap': 0.5,
    'ygap': 0.5,

    # Colors and fonts
    'plot_bg_color': 'white',
    'paper_bg_color': 'white',
    'font_color': 'black',
    'base_font_size': 10,

    # Colorbar and labels positions
    'colorbar': {'x': 1.14, 'thickness': 15},
    'colorbar_label': {'text': 'Profit % Change', 'x': 1.22, 'angle': 90},
    'v6_axis_label': {
        'text': 'Proportion of Cultivars Susceptible to V6-virulent Strains',
        'x': 1.09,
        'angle': 90,
    },
    'percent_v6_numbers_x': 1.015,

    # Demand and percentile labels
    'demand_label': {'x': -0.07, 'angle': -90, 'font_size': 10},
    'percentile_label': {'x': -0.11, 'text': 'Percentile of Dispersal-Centrality', 'angle': -90, 'font_size': 12},

    # Column headers
    'p0_label': {'y': 1.05, 'font_size': 11, 'scientific': True, 'sig_figs': 2},
    'p0_title': {'y': 1.11, 'font_size': 12, 'text': 'Initial Probability of Disease'},

    # Shared x-axis title
    'x_title': {'y': -0.09, 'font_size': 12, 'text': 'Number of Sprays in May'},

    # Tick styles (outer axes only)
    'ticks': {
        'len': 3,
        'width': 1,
        'color': 'rgba(0,0,0,0.4)',
        'font_size': 8,
        'outer_only': True,
        'min_mid_max': True,
    },

    # Brackets near demand labels
    'brackets': {
        'x_inner': -0.058,
        'x_outer': -0.045,
        'gap': 0.01,
        'line_color': 'rgba(0,0,0,0.9)',
        'line_width': 1.2,
    },

    # Outside tick marker lines
    'outside_ticks': {
        'top_height': 0.012,
        'right_width': 0.008,
        'color': 'rgba(0,0,0,0.35)',
        'width': 1,
    },

    # Boxes for specific labels (bg and border for descriptive labels only)
    'box_style': {
        'bg': 'rgba(255,255,255,0.9)',
        'border_color': 'rgba(0,0,0,0.25)',
        'border_width': 0.6,
        'pad': 2,
    },

    # Indicator controls
    'indicator': {
        'enabled': True,           # toggle on/off
        'style': 'dot',            # one of: 'dot', 'ring', 'cross'
        'size': 4,                 # base size (smaller default)
        # base colors; some styles will override
        'fill': 'rgba(255,255,255,0.9)',
        'border_color': 'rgba(0,0,0,0.85)',
        'border_width': 0.6,
    },

    # Export options
    'export': {
        'html': False,
        'png': True,               # enable PNG export
        'png_scale': 3,            # higher scale for finer rasterization (try 3-4)
        'file_prefix': 'profit_heatmap_all_demands',
        'output_dir': '../reports/figures'  # output directory
    },

    # Modebar download options (controls the quality of the camera icon PNG)
    'modebar': {
        'to_image': {
            'format': 'svg',     # 'png', 'svg', 'jpeg', 'webp'
            'scale': 6,          # multiply current figure resolution (try 2-4)
            # width/height are optional; if omitted, uses figure size * scale
            # 'width': 1200,
            # 'height': 900,
            # 'filename' cannot be set from the modebar config; browser names it
        }
    }
}


In [7]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

# Pull configuration with defaults if cfg is not defined
try:
    _cfg = cfg
except NameError:
    _cfg = {}

# Define parameters
year = _cfg.get('year', 2014)  # 2014, 2015, 2016, 2017
# We'll include all market demand levels stacked vertically (high, moderate, low)

# --- Step 1: Read the full data for the year to get global min/max ---
file_name = _cfg.get('file_name', '../data/processed/simulations/simulations_apr7.parquet')
df_full_year = pd.read_parquet(file_name)
df_full_year = df_full_year[df_full_year['Year'] == year]

# --- Step 2: Determine global color scale limits for the entire year ---
v_min = df_full_year['Mean Profit Percent Change'].min()
v_max = df_full_year['Mean Profit Percent Change'].max()
print(f"Global Min/Max for {year}: {v_min:.2f}%, {v_max:.2f}%")  # Optional: print the range

# --- Step 3: Keep ALL market demand levels (no filtering) ---
df_profit_all = df_full_year.copy()
# Normalize demand names to lowercase for consistent filtering
md_lower = df_profit_all['Market Demand'].astype(str).str.lower()
df_profit_all = df_profit_all.assign(_md_lower=md_lower)

# Define demand level order (top to bottom). Only include those present in the data.
preferred_order = ['high', 'moderate', 'low']
available = list(sorted(df_profit_all['_md_lower'].unique()))
demand_levels = [d for d in preferred_order if d in available] + [d for d in available if d not in preferred_order]

# Get sorted unique V6 Percent and Initial Probability values across all demands
unique_v6_percent = sorted(df_profit_all['V6 Percent'].unique())
unique_init_prob = sorted(df_profit_all['Initial Probability'].unique())

# --- Initial Probability labels: base-10 scientific or decimal (cfg-controlled) ---
use_sci = _cfg.get('p0_label', {}).get('scientific', False)
sig_figs = int(_cfg.get('p0_label', {}).get('sig_figs', 2))

def _to_sci_base10(val: float, sig: int) -> str:
    if val == 0 or np.isclose(val, 0):
        return '0'
    exp = int(np.floor(np.log10(abs(val))))
    mant = val / (10 ** exp)
    # Round mantissa to requested significant figures
    fmt = f"{{:.{max(sig-1,0)}f}}"
    mant_s = fmt.format(mant).rstrip('0').rstrip('.')
    # Handle rounding edge-case like 9.99 -> 10.0
    if mant_s in ('10', '10.0'):
        mant_s = '1'
        exp += 1
    supers = str.maketrans({'0':'⁰','1':'¹','2':'²','3':'³','4':'⁴','5':'⁵','6':'⁶','7':'⁷','8':'⁸','9':'⁹','-':'⁻'})
    exp_s = str(exp).translate(supers)
    return f"{mant_s}×10{exp_s}"

def _format_p0(val: float) -> str:
    return _to_sci_base10(float(val), sig_figs) if use_sci else (f"{val:.5f}".rstrip('0').rstrip('.') or '0')

unique_init_prob_labels = [_format_p0(prob) for prob in unique_init_prob]

# Define the grid size (stack each demand's V6 rows vertically)
v6_rows = len(unique_v6_percent)
n_rows = len(demand_levels) * v6_rows
n_cols = len(unique_init_prob)

# Margins and spacing
_margins = _cfg.get('margins', {'l': 110, 'r': 210, 't': 130, 'b': 90})
left_margin = _margins.get('l', 110)
right_margin = _margins.get('r', 210)  # nudged to give extra room on the right
top_margin = _margins.get('t', 130)
bottom_margin = _margins.get('b', 90)
# Keep tiny spacing to minimize compression
_spacing = _cfg.get('spacing', {'horizontal': 0.002, 'vertical': 0.002})
horizontal_spacing = _spacing.get('horizontal', 0.002)
vertical_spacing = _spacing.get('vertical', 0.002)

# Create subplot titles as empty strings
subplot_titles = [''] * (n_rows * n_cols)

# Create the figure with subplots
fig = make_subplots(
    rows=n_rows, cols=n_cols,
    subplot_titles=subplot_titles,
    horizontal_spacing=horizontal_spacing,
    vertical_spacing=vertical_spacing,
    shared_xaxes=True,
    shared_yaxes=True,
    specs=[[{'type': 'heatmap'} for _ in range(n_cols)] for _ in range(n_rows)]
)

# --- Step 4: Use a colorblind-friendly colorscale ---
colorscale_choice = _cfg.get('colorscale', 'Viridis')  # Other options: 'Plasma', 'Magma', 'Viridis'

# Add heatmaps to subplots for each (demand, V6, initial_prob) combination
# Also track one example pivot to infer x/y category counts for square cells
example_nx = None
example_ny = None
for d_idx, demand in enumerate(demand_levels):
    for v_idx, v6_value in enumerate(unique_v6_percent):
        # Compute the destination row group for this (demand, v6)
        base_row = d_idx * v6_rows + v_idx + 1
        # Subset once per (demand, v6)
        subset_dv6 = df_profit_all[(df_profit_all['_md_lower'] == demand) & (df_profit_all['V6 Percent'] == v6_value)]
        if subset_dv6.empty:
            continue
        for init_prob_value in unique_init_prob:
            df_filtered = subset_dv6[subset_dv6['Initial Probability'] == init_prob_value]
            if df_filtered.empty:
                # Skip if no data for this combination
                continue
            # Pivot the data for heatmap
            pivot_profit = df_filtered.pivot_table(
                values='Mean Profit Percent Change',
                index='Quantile',
                columns='Sprays in May',
                aggfunc='mean'
            ).sort_index(axis=1)
            if pivot_profit.empty:
                continue
            # Get x and y axes labels
            x_labels = pivot_profit.columns.tolist()
            y_labels = pivot_profit.index.tolist()  # Quantiles (assumed consistent across subplots)
            # Capture example sizes
            if example_nx is None:
                example_nx = len(x_labels)
            if example_ny is None:
                example_ny = len(y_labels)
            # Determine subplot position
            col = unique_init_prob.index(init_prob_value) + 1
            # Add heatmap to the subplot
            fig.add_trace(
                go.Heatmap(
                    z=pivot_profit.values,
                    x=x_labels,
                    y=y_labels,
                    colorscale=colorscale_choice,
                    # --- Apply Global Min/Max ---
                    zmin=v_min,
                    zmax=v_max,
                    coloraxis='coloraxis',
                    showscale=False,
                    xgap=_cfg.get('xgap', 0.5),  # equal gaps to keep square look
                    ygap=_cfg.get('ygap', 0.5),
                    hoverongaps=False
                ),
                row=base_row,
                col=col
            )

# --- Step 5: Set white background and black font ---
plot_bg_color = _cfg.get('plot_bg_color', 'white')
paper_bg_color = _cfg.get('paper_bg_color', 'white')
font_color = _cfg.get('font_color', 'black')
base_font_size = _cfg.get('base_font_size', 10)

# Update layout (without forcing huge width/height yet)
fig.update_layout(
    template='plotly_white',  # Use a white template as a base
    plot_bgcolor=plot_bg_color,
    paper_bgcolor=paper_bg_color,
    font=dict(
        color=font_color,
        size=base_font_size
    ),
    title=dict(
        text=f'Relative Change in Profit across Market Demand Levels in {year}',
        font=dict(
            color=font_color,
            size=18
        ),
        x=0.5,
        y=0.98,
        xanchor='center',
        yanchor='top'
    ),
    margin=dict(l=left_margin, r=right_margin, t=top_margin, b=bottom_margin),
    # --- Apply Global Min/Max to coloraxis ---
    coloraxis=dict(
        colorscale=colorscale_choice,
        cmin=v_min,
        cmax=v_max,
        colorbar=dict(
            title=dict(
                text='',  # we'll render a rotated label via annotation instead
                side='right',
                font=dict(color=font_color)
            ),
            tickfont=dict(size=10, color=font_color),
            len=1.0,
            y=0.5,
            x=_cfg.get('colorbar', {}).get('x', 1.14),  # move colorbar a bit right
            thickness=_cfg.get('colorbar', {}).get('thickness', 15)
        )
    ),
    shapes=[]  # We'll set shapes below
)

# Disable axis gridlines across all axes to remove faint lines through cells
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

# Remove axis tick marks and axis lines initially (we'll re-enable with custom settings below)
fig.update_xaxes(ticks="", ticklen=0, tickwidth=0, tickcolor="rgba(0,0,0,0)", showline=False, zeroline=False, mirror=False)
fig.update_yaxes(ticks="", ticklen=0, tickwidth=0, tickcolor="rgba(0,0,0,0)", showline=False, zeroline=False, mirror=False)

# SWAP POSITIONS:
# 1) Place Demand level labels (Low/Moderate/High) inward where the Percentile label was (left-inside)
_demand_label = _cfg.get('demand_label', {'x': -0.07, 'angle': -90, 'font_size': 10})
for d_idx, demand in enumerate(demand_levels):
    y_center = 1 - (((d_idx * v6_rows) + (v6_rows / 2)) / n_rows)
    fig.add_annotation(
        dict(
            text=demand.capitalize(),
            x=_demand_label.get('x', -0.07),  # where the Percentile title was
            y=y_center,
            xref='paper',
            yref='paper',
            textangle=_demand_label.get('angle', -90),  # rotate demand labels vertically
            showarrow=False,
            font=dict(size=_demand_label.get('font_size', 10), color=font_color),  # reduced font size
            align='center',
            xanchor='right',
            yanchor='middle'
        )
    )

# 2) Move Percentile of Dispersal-Centrality to where the demand labels were (outer-left), rotated
_percentile_label = _cfg.get('percentile_label', {'x': -0.11, 'text': 'Percentile of Dispersal-Centrality', 'angle': -90, 'font_size': 12})
_box_style = _cfg.get('box_style', {'bg': 'rgba(255,255,255,0.9)', 'border_color': 'rgba(0,0,0,0.25)', 'border_width': 0.6, 'pad': 2})
fig.add_annotation(
    dict(
        text=_percentile_label.get('text', 'Percentile of Dispersal-Centrality'),
        x=_percentile_label.get('x', -0.11),  # where demand labels were
        y=0.5,
        textangle=_percentile_label.get('angle', -90),
        xref='paper',
        yref='paper',
        showarrow=False,
        font=dict(size=_percentile_label.get('font_size', 12), color=font_color),
        align='center',
        xanchor='right',
        yanchor='middle',
        bgcolor=_box_style.get('bg', 'rgba(255,255,255,0.9)'),  # add box around label
        bordercolor=_box_style.get('border_color', 'rgba(0,0,0,0.25)'),
        borderwidth=_box_style.get('border_width', 0.6),
        borderpad=_box_style.get('pad', 2)
    )
)

# Add initial probability labels (top) WITHOUT boxes
_p0_label = _cfg.get('p0_label', {'y': 1.05, 'font_size': 11})
for idx, init_prob_label in enumerate(unique_init_prob_labels):
    x_pos = (idx + 1 - 0.5) / n_cols
    fig.add_annotation(
        dict(
            text=f"{init_prob_label}",
            x=x_pos,
            y=_p0_label.get('y', 1.05),
            xref='paper',
            yref='paper',
            showarrow=False,
            font=dict(size=_p0_label.get('font_size', 11), color=font_color),
            align='center',
            xanchor='center'
        )
    )

# Top-center title for columns WITH a box
_p0_title = _cfg.get('p0_title', {'y': 1.13, 'font_size': 12, 'text': 'Initial Probability of Disease'})
fig.add_annotation(
    dict(
        text=_p0_title.get('text', 'Initial Probability of Disease'),
        x=0.5,
        y=_p0_title.get('y', 1.11),
        xref='paper',
        yref='paper',
        showarrow=False,
        font=dict(size=_p0_title.get('font_size', 12), color=font_color),
        align='center',
        xanchor='center',
        bgcolor=_box_style.get('bg', 'rgba(255,255,255,0.9)'),
        bordercolor=_box_style.get('border_color', 'rgba(0,0,0,0.25)'),
        borderwidth=_box_style.get('border_width', 0.6),
        borderpad=_box_style.get('pad', 2)
    )
)

# Row labels as proportions (V6) WITHOUT boxes, and a reflected/rotated axis label slightly to the right
percent_v6_x_pos = _cfg.get('percent_v6_numbers_x', 1.015)  # where the numerical labels sit
axis_label_x_pos = _cfg.get('v6_axis_label', {}).get('x', 1.09)    # further right for the rotated long-form label (moved right)

for d_idx, _d in enumerate(demand_levels):
    for v_idx, v6_value in enumerate(unique_v6_percent):
        row_index = d_idx * v6_rows + v_idx
        y_pos = 1 - ((row_index + 0.5) / n_rows)
        prop_label = ('{:.2f}'.format(float(v6_value))).rstrip('0').rstrip('.')
        fig.add_annotation(
            dict(
                text=prop_label,
                x=percent_v6_x_pos,
                y=y_pos,
                xref='paper',
                yref='paper',
                showarrow=False,
                font=dict(size=11, color=font_color),
                align='right',
                xanchor='left',
                yanchor='middle'
            )
        )

# Rotated long-form axis label (reflected top-to-bottom)
_v6_axis_label = _cfg.get('v6_axis_label', {'text': 'Proportion of Cultivars Susceptible to V6-virulent Strains', 'x': 1.09, 'angle': 90})
fig.add_annotation(
    dict(
        text=_v6_axis_label.get('text', 'Proportion of Cultivars Susceptible to V6-virulent Strains'),
        x=_v6_axis_label.get('x', 1.09),
        y=0.5,
        xref='paper',
        yref='paper',
        textangle=_v6_axis_label.get('angle', 90),  # vertical label, top-to-bottom
        showarrow=False,
        font=dict(size=12, color=font_color),
        align='center',
        xanchor='left',
        yanchor='middle',
        bgcolor=_box_style.get('bg', 'rgba(255,255,255,0.9)'),
        bordercolor=_box_style.get('border_color', 'rgba(0,0,0,0.25)'),
        borderwidth=_box_style.get('border_width', 0.6),
        borderpad=_box_style.get('pad', 2)
    )
)

# Add a reflected (vertical) label for the colorbar instead of the built-in title (NO box)
_cbar_label = _cfg.get('colorbar_label', {'text': 'Profit % Change', 'x': 1.22, 'angle': 90})
fig.add_annotation(
    dict(
        text=_cbar_label.get('text', 'Profit % Change'),
        x=_cbar_label.get('x', 1.22),  # to the right of the colorbar
        y=0.5,
        xref='paper',
        yref='paper',
        textangle=_cbar_label.get('angle', 90),  # vertical label, top-to-bottom
        showarrow=False,
        font=dict(size=12, color=font_color),
        align='center',
        xanchor='left',
        yanchor='middle'
    )
)

# Add left-side bracket shapes close to the demand labels with small gaps between blocks
shapes = []
# Bracket geometry (in paper coordinates, near the labels)
_br = _cfg.get('brackets', {'x_inner': -0.058, 'x_outer': -0.045, 'gap': 0.01, 'line_color': 'rgba(0,0,0,0.9)', 'line_width': 1.2})
x_inner = _br.get('x_inner', -0.058)   # just to the right of the demand labels
x_outer = _br.get('x_outer', -0.045)   # short arm length toward the plot
line_style = dict(color=_br.get('line_color', 'rgba(0,0,0,0.9)'), width=_br.get('line_width', 1.2))  # black, thinner
gap = _br.get('gap', 0.01)  # small vertical gap between brackets
for d_idx in range(len(demand_levels)):
    y_top = 1 - (d_idx * v6_rows) / n_rows
    y_bottom = 1 - ((d_idx + 1) * v6_rows) / n_rows
    # Shrink vertical span to create gaps so adjacent brackets don't touch
    if d_idx == 0:
        y0 = y_bottom + gap
        y1 = y_top
    elif d_idx == len(demand_levels) - 1:
        y0 = y_bottom
        y1 = y_top - gap
    else:
        y0 = y_bottom + gap
        y1 = y_top - gap
    # Vertical segment
    shapes.append(dict(type='line', xref='paper', yref='paper',
                       x0=x_inner, x1=x_inner, y0=y0, y1=y1,
                       line=line_style))
    # Top short horizontal segment
    shapes.append(dict(type='line', xref='paper', yref='paper',
                       x0=x_inner, x1=x_outer, y0=y1, y1=y1,
                       line=line_style))
    # Bottom short horizontal segment
    shapes.append(dict(type='line', xref='paper', yref='paper',
                       x0=x_inner, x1=x_outer, y0=y0, y1=y0,
                       line=line_style))

# Add subtle outside ticks above each column center to reinforce p0 alignment
_out_ticks = _cfg.get('outside_ticks', {'top_height': 0.012, 'right_width': 0.008, 'color': 'rgba(0,0,0,0.35)', 'width': 1})
for idx in range(n_cols):
    x_center = (idx + 0.5) / n_cols
    shapes.append(dict(type='line', xref='paper', yref='paper',
                       x0=x_center, x1=x_center, y0=1.0, y1=1.0 + _out_ticks.get('top_height', 0.012),
                       line=dict(color=_out_ticks.get('color', 'rgba(0,0,0,0.35)'), width=_out_ticks.get('width', 1))))

# Add subtle outside ticks to the right of each row center to reinforce alignment
for row_index in range(n_rows):
    y_center = 1 - ((row_index + 0.5) / n_rows)
    shapes.append(dict(type='line', xref='paper', yref='paper',
                       x0=1.0, x1=1.0 + _out_ticks.get('right_width', 0.008), y0=y_center, y1=y_center,
                       line=dict(color=_out_ticks.get('color', 'rgba(0,0,0,0.35)'), width=_out_ticks.get('width', 1))))

# Subtle outer border and boxes around specific headings
shapes.append(dict(type='rect', xref='paper', yref='paper', x0=0, y0=0, x1=1, y1=1,
                   line=dict(color='rgba(0,0,0,0.15)', width=1)))

fig.update_layout(shapes=shapes)

# Small ticks and reduced tick-label font on outer axes only
# Choose a small, interpretable set: min, mid, max
x_levels = sorted(df_profit_all['Sprays in May'].unique())
y_levels = sorted(df_profit_all['Quantile'].unique())

def _pick_ticks(levels):
    try:
        vals = sorted(set(float(v) for v in levels))
    except Exception:
        # Fallback if not numeric
        return list(levels)[:3]
    if len(vals) <= 3:
        return vals
    minv, maxv = vals[0], vals[-1]
    mid_target = (minv + maxv) / 2
    midv = min(vals, key=lambda v: abs(v - mid_target))
    return [minv, midv, maxv]

x_tickvals_num = _pick_ticks(x_levels)
y_tickvals_num = _pick_ticks(y_levels)

# Format text; use strings as tickvals to be safe on categorical axes
x_ticktext = [str(int(v)) if float(v).is_integer() else ('{:.2f}'.format(v)).rstrip('0').rstrip('.') for v in x_tickvals_num]
y_ticktext = [('{:.2f}'.format(v)).rstrip('0').rstrip('.') for v in y_tickvals_num]

# Set base tick appearance across axes
_ticks = _cfg.get('ticks', {'len': 3, 'width': 1, 'color': 'rgba(0,0,0,0.4)', 'font_size': 8, 'outer_only': True, 'min_mid_max': True})
fig.update_xaxes(
    tickmode='array', tickvals=x_ticktext, ticktext=x_ticktext,
    ticks='outside', ticklen=_ticks.get('len', 3), tickwidth=_ticks.get('width', 1), tickcolor=_ticks.get('color', 'rgba(0,0,0,0.4)'),
    tickfont=dict(size=_ticks.get('font_size', 8), color=font_color)
)
fig.update_yaxes(
    tickmode='array', tickvals=y_ticktext, ticktext=y_ticktext,
    ticks='outside', ticklen=_ticks.get('len', 3), tickwidth=_ticks.get('width', 1), tickcolor=_ticks.get('color', 'rgba(0,0,0,0.4)'),
    tickfont=dict(size=_ticks.get('font_size', 8), color=font_color)
)

# Now limit visibility: show y ticks/labels only on the first column; x ticks/labels only on the last row
for r in range(1, n_rows + 1):
    for c in range(1, n_cols + 1):
        show_y = (c == 1)
        show_x = (r == n_rows)
        fig.update_yaxes(row=r, col=c,
                         showticklabels=show_y,
                         ticks=('outside' if show_y else ''),
                         ticklen=(_ticks.get('len', 3) if show_y else 0),
                         tickwidth=(_ticks.get('width', 1) if show_y else 0))
        fig.update_xaxes(row=r, col=c,
                         showticklabels=show_x,
                         ticks=('outside' if show_x else ''),
                         ticklen=(_ticks.get('len', 3) if show_x else 0),
                         tickwidth=(_ticks.get('width', 1) if show_x else 0))

# Add shared x-axis title (keep centered) WITH a box
_x_title = _cfg.get('x_title', {'y': -0.09, 'font_size': 12, 'text': 'Number of Sprays in May'})
fig.add_annotation(
    dict(
        text=_x_title.get('text', 'Number of Sprays in May'),
        x=0.5,
        y=_x_title.get('y', -0.09),
        xref='paper',
        yref='paper',
        showarrow=False,
        font=dict(size=_x_title.get('font_size', 12), color=font_color),
        align='center',
        bgcolor=_box_style.get('bg', 'rgba(255,255,255,0.9)'),
        bordercolor=_box_style.get('border_color', 'rgba(0,0,0,0.25)'),
        borderwidth=_box_style.get('border_width', 0.6),
        borderpad=_box_style.get('pad', 2)
    )
)

# Make little heatmap boxes square by adjusting the figure height vs width (use previous approach)
if example_nx is None:
    example_nx = df_profit_all['Sprays in May'].nunique()
if example_ny is None:
    example_ny = df_profit_all['Quantile'].nunique()
target_width = int(_cfg.get('target_width', 1000))  # slightly wider for the extra right-side labels
avail_width = target_width - left_margin - right_margin
height_available = avail_width * (n_rows * example_ny) / (n_cols * example_nx)
target_height = int(height_available + top_margin + bottom_margin)
fig.update_layout(width=target_width, height=target_height)

# Update hover template
fig.update_traces(
    hovertemplate=
    'Sprays in May: %{x}<br>' +
    'Dispersal Percentile: %{y}<br>' +
    'Profit % Change: %{z:.2f}%<extra></extra>'
)

# Configure high-res modebar download
_to_image_cfg = _cfg.get('modebar', {}).get('to_image', {})
modebar_config = {
    'toImageButtonOptions': {
        'format': _to_image_cfg.get('format', 'png'),
        'scale': int(_to_image_cfg.get('scale', 3)),
        # width/height optional; if not provided, Plotly multiplies current figure size by scale
        **({ 'width': int(_to_image_cfg['width']) } if 'width' in _to_image_cfg else {}),
        **({ 'height': int(_to_image_cfg['height']) } if 'height' in _to_image_cfg else {}),
    }
}

# Show the figure (with hi-res download button)
fig.show(config=modebar_config)

# Optional: Save the figure
_export = _cfg.get('export', {'html': False, 'png': False, 'png_scale': 3, 'file_prefix': 'profit_heatmap_all_demands', 'output_dir': '../reports/figures'})
_out_dir = _export.get('output_dir', '.')
os.makedirs(_out_dir, exist_ok=True)
_fname_base = f"{_out_dir}/" + f"{_export.get('file_prefix', 'profit_heatmap_all_demands')}_{year}_white_{colorscale_choice}_global_scale"
if _export.get('html', False):
    fig.write_html(f'{_fname_base}.html')
if _export.get('png', False):
    # Requires kaleido installed: pip install -U kaleido
    fig.write_image(f'{_fname_base}.png', scale=int(_export.get('png_scale', 3)))
# (Optional) Save a vector version for journals
if _export.get('pdf', False):
    fig.write_image(f'{_fname_base}.pdf')
if _export.get('svg', False):
    fig.write_image(f'{_fname_base}.svg')


Global Min/Max for 2017: -8.39%, 0.39%
